# Assignment 4
## Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=400>

In this quiz, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so we will account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into pandas dataframes called bus and ins respectively. Print the first 5 rows of each to inspect them.


In [15]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

bus = pd.read_csv("data/businesses.csv", encoding='ISO-8859-1')
ins = pd.read_csv("data/inspections.csv")

In [16]:
bus.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [17]:
ins.head(5)

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [18]:
#SOLUTION CELL
print("Number of records:", len(bus))
#or
print("Number of records:", bus.shape[0])


Number of records: 6315
Number of records: 6315


#### Question 2b: How many unique business IDs are there?  

In [19]:
print("Number of unique business ids:", len(bus['business_id'].unique()))

Number of unique business ids: 6315


#### Question 2c: is the `business_id` unique in this data?

In [20]:
#SOLUTION CELL
bus['business_id'].value_counts().max() == 1


True

#### Question 2d: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [21]:
#SOLUTION CELL
print("Most frequently occuring business names:")
print(bus['name'].value_counts()[:5])

Most frequently occuring business names:
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values encoded in python: ints, floats, strings, booleans ...

To answer this you might want to examine a particular entry.

In [22]:
#SOLUTION CELL
print('the data type for the indiviudual elements of postal code is: ')
type(bus['postal_code'][0])

the data type for the indiviudual elements of postal code is: 


str

In [23]:
print('Note that this is different than the dtype of the Pandas Column:')
print(bus['postal_code'].dtype)

Note that this is different than the dtype of the Pandas Column:
object


#### Question 3b

What are the unique values of postal_code?

In [24]:
#SOLUTION CELL
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [25]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [26]:
#SOLUTION CELL
bus_sf = bus[bus['postal_code'].isin(validZip)]
print(bus_sf.shape[0], "businesses")

5999 businesses


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are represenative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude and latitude values, working with only the businesses that are in the list of valid zipcodes?

In [27]:
#SOLUTION CELL
print(len(bus_sf[bus_sf['latitude'].isnull()]),'missing latitude values')
print(len(bus_sf[bus_sf['longitude'].isnull()]),'missing longitude values')

2483 missing latitude values
2483 missing longitude values


#### Question 4b

For these zip codes find the number of businesses in each zip code and the number without longitude values. 

Create a new dataframe of counts of the null and proportion of null values, storing the result in `bus_sf_latlong`. It should have 3 columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of missing values for the zip code.
3. `not_null_lon`: The number of present values for the zip code.

In [28]:
#SOLUTION CELL

total = bus_sf['postal_code'].groupby(bus_sf['postal_code']).count().to_frame(name='total').reset_index()

nulls = bus_sf[bus_sf['longitude'].isnull()]
null_lon = nulls['postal_code'].groupby(nulls['postal_code']).count().to_frame(name='null_lon').reset_index()

nonulls = bus_sf[bus_sf['longitude'].notnull()]
not_null_lon = nonulls['postal_code'].groupby(nonulls['postal_code']).count().to_frame(name='not_null_lon').reset_index()

merge1 = pd.merge(total, null_lon, on='postal_code', how='outer')
bus_sf_latlong = pd.merge(merge1, not_null_lon, on='postal_code', how='outer')

bus_sf_latlong.head()

,postal_code,total,null_lon,not_null_lon
0,94102,458,196,262
1,94103,558,252,306
2,94104,133,59,74
3,94105,226,100,126
4,94107,451,247,204


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [29]:
#SOLUTION CELL
bus_sf_latlong['share_missing'] = bus_sf_latlong['null_lon'] / bus_sf_latlong['total'] * 100

bus_sf_latlong.sort_values(by='share_missing', ascending = False).head()

,postal_code,total,null_lon,not_null_lon,share_missing
4,94107,451,247,204,54.767184
18,94124,164,88,76,53.658537
21,94132,134,65,69,48.507463
10,94114,230,108,122,46.956522
16,94122,270,123,147,45.555556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [30]:
# SOLUTION CELL 
print('rows in table', ins.shape[0])
print('number of unique business_ids',len(ins['business_id'].unique()))

# So there are more than one inspection per every business. 

rows in table 15430
number of unique business_ids 5730


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe ins2, keeping from the dataframe only records with values of `type` that occur more than 10 times in the inspections file. (eliminate records that have values of `type` that occur rarely). Check the result to make sure rare types are eliminated.

In [31]:
pd.value_counts(ins['type'])

routine      15429
complaint        1
Name: type, dtype: int64

In [32]:
ins2 = ins[ins['type']!='complaint']
pd.value_counts(ins2['type'])

routine    15429
Name: type, dtype: int64

#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

BIG Hint: you could apply a lambda function here to convert the values to a string and slice off the first four characters.

In [33]:
ins2.loc[:,'year'] = ins['date'].apply(lambda y: str(y)[:4])
ins2.head()

,business_id,score,date,type,year
0,10,82,20160503,routine,2016
1,10,94,20140729,routine,2014
2,10,92,20140114,routine,2014
3,19,94,20160513,routine,2016
4,19,94,20141110,routine,2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year?  Maybe you should drop records for any years with less than 50 inspections to avoid biasing an analysis.  Do that to create a new dataframe called ins3.

In [34]:
pd.value_counts(ins2['year'])

2016    5839
2014    5629
2015    3923
2013      38
Name: year, dtype: int64

In [35]:
ins3 = ins2[ins2['year']>'2013']
pd.value_counts(ins3['year'])

2016    5839
2014    5629
2015    3923
Name: year, dtype: int64

Let's examine only the inspections for one year, 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [36]:
ins2016 = ins3[ins3['year']=='2016']
ins2016.shape[0]

5839

### Question 6

#### Question 6a

Merge the business and 2016 inspections data first. 

In [37]:
final = pd.merge(bus_sf,ins2016, on='business_id', how='left')
final.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503.0,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513.0,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005.0,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311.0,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816.0,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses and their ratings.

In [38]:
final[['name','address','score']].sort_values(by='score').head(20)

,name,address,score
4755,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
4736,UNCLE CAFE,65 WAVERLY PL,55.0
6042,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
7020,POKI TIME,2101 LOMBARD ST,55.0
5448,GOLDEN WOK,295 B ORIZABA AVE,56.0
2968,HING WANG BAKERY,339 JUDAH ST,56.0
836,RED A BAKERY,634 CLEMENT ST,57.0
5861,L & G VIETNAMESE SANDWICH,602 EDDY ST,57.0
4107,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0
4766,"NEW GARDEN RESTAURANT, INC.",716 KEARNY ST,57.0


#### Question 6c: what is the mean rating in the lowest rated 5 postal codes?

In [39]:
final.groupby('postal_code')['score'].mean().sort_values().head()

postal_code
94121    86.121387
94134    87.421053
94118    87.516807
94109    87.765000
94133    88.081081
Name: score, dtype: float64

## Done!

Now submit your notebook to the bcourses site.